In [31]:
import pandas as pd

In [32]:
import googlemaps

In [58]:
allway_points = ["Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA",
                 "Death Valley National Park, Furnace Creek, Inyo County, California",
                 "Kohm Yah-mah-nee Visitor Center, Lassen Peak Highway, Mineral, CA",
                 "Redwood National and State Parks Visitor Center, U.S. 199, Crescent City, CA",
                 "Pinnacles Visitor Center, California 146, Paicines, CA",
                 "Mount Rainier National Park Headquarters, 238th Avenue East, Ashford, WA",
                 "Glacier National Park Headquarters, Grinnell Drive, West Glacier, MT",
                 "Yellowstone National Park Headquarters, Yellowstone National Park, WY",
                 "Rocky Mountain National Park Grand Lake Entrance, Trail Ridge Road, Grand Lake, CO",
                 "Zion National Park South Entrance Ranger Station, Springdale, UT",
                 "Grand Canyon North Rim Lodge, Arizona 67, North Rim, AZ",
                 "Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT",
                 "Shenandoah National Park Headquarters, Luray, VA",
                 "Shenandoah National Park, Virginia",
                 "Carlsbad Caverns National Park Visitor Center, Carlsbad Cavern Highway, Carlsbad, New Mexico",
                 "Grand Teton National Park Headquarters, Moose, WY"]

In [59]:
gmaps = googlemaps.Client(key="xxxxxxx")

In [60]:
from itertools import combinations
waypoint_distances={}
waypoint_durations={}
for (waypoint1,waypoint2) in combinations(allway_points,2):
    route= gmaps.distance_matrix(origins=[waypoint1],
                                 destinations=[waypoint2],
                                 mode="driving",
                                 language="English",
                                 units="metric")
    distance = route["rows"][0]["elements"][0]["distance"]["value"]
    duration = route["rows"][0]["elements"][0]["duration"]["value"]
    
    waypoint_distances[frozenset([waypoint1,waypoint2])] = distance
    waypoint_durations[frozenset([waypoint1,waypoint2])] = duration
    

In [61]:
print route["rows"][0]["elements"][0]["duration"]["value"]

61166


In [62]:
with open("loc_dist_dur.csv","w") as out_file:
    out_file.write("\t".join(["waypoint1","waypoint2","distance","duration"]))
    
    for (waypoint1,waypoint2) in waypoint_distances.keys():
        out_file.write("\n"+
                       "\t".join([waypoint1,waypoint2,str(waypoint_distances[frozenset([waypoint1,waypoint2])]),str(waypoint_durations[frozenset([waypoint1,waypoint2])])]))

In [63]:
all_waypoints = set()
waypoint_distances={}
waypoint_durations={}
waypoint_data = pd.read_csv("loc_dist_dur.csv",sep="\t")

In [64]:
waypoint_data.head()

,waypoint1,waypoint2,distance,duration
0,"Shenandoah National Park, Virginia","Yosemite Valley Visitor Center, Village Drive,...",4217636,141372
1,"Carlsbad Caverns National Park Visitor Center,...",Redwood National and State Parks Visitor Cente...,2711579,91021
2,"Mount Rainier National Park Headquarters, 238t...","Yellowstone National Park Headquarters, Yellow...",1257768,45065
3,"Grand Teton National Park Headquarters, Moose, WY","Yellowstone National Park Headquarters, Yellow...",217608,11921
4,"Kohm Yah-mah-nee Visitor Center, Lassen Peak H...","Yellowstone National Park Headquarters, Yellow...",1469950,52296


In [65]:
for i,row in waypoint_data.iterrows():
    waypoint_distances[frozenset([row.waypoint1,row.waypoint2])]=row.distance
    waypoint_durations[frozenset([row.waypoint1,row.waypoint2])]=row.duration
    all_waypoints.update([row.waypoint1,row.waypoint2])

In [66]:
import random

In [67]:
def compute_fitness(agent_gnome):
    fitness =0.0
    for index in range(len(agent_gnome)):
        fitness += waypoint_distances[frozenset([agent_gnome[index-1],agent_gnome[index]])]
    return fitness
        

def generate_random_agent():
    random_agent = list(all_waypoints)
    random.shuffle(random_agent)
    return tuple(random_agent)

def generate_random_population(population_size):
    random_population=[]
    for i in range(population_size):
        random_population.append(generate_random_agent())
    return random_population

def mutate(agent_gnome,number_of_swaps):
    agent_gnome = list(agent_gnome)
    number_swap = random.randint(0,number_of_swaps)
    
    for swap in range(number_swap):
        swap_index1 = random.randint(0,len(agent_gnome)-1)
        swap_index2 = swap_index1
        while swap_index2 == swap_index1:
            swap_index2 = random.randint(0,len(agent_gnome)-1)
        agent_gnome[swap_index1],agent_gnome[swap_index2] = agent_gnome[swap_index2],agent_gnome[swap_index1]
        
    return tuple(agent_gnome)

def shuffle_mutate(agent_gnome):
    agent_gnome = list(agent_gnome)
    
    start_index = random.randint(0,len(agent_gnome)-1)
    length = random.randint(2,10)
    
    genome_subset = agent_gnome[start_index:start_index + length]
    agent_genome = agent_gnome[:start_index] + agent_gnome[start_index + length:]
    
    insert_index = random.randint(0, len(agent_gnome) + len(genome_subset) - 1)
    agent_gnome = agent_gnome[:insert_index] + genome_subset + agent_gnome[insert_index:]
    
    return tuple(agent_gnome)
    
    
        



In [68]:
population=[]
def genetic_algorithm(generations=5000,population_size=1000):
    population_subset_size = int (population_size/10)
    population = generate_random_population(population_size)
    for generation in range(generations):
        population_fitness={}
        for agent_gnome in population:
            if agent_gnome in population_fitness:
                continue
            population_fitness[agent_gnome]=compute_fitness(agent_gnome)
        new_population = []
        for rank,agent_gnome in enumerate(sorted(population_fitness,key=population_fitness.get)[:population_subset_size]):
            new_population.append(agent_gnome)
            
            for i  in range(2):
                new_population.append(mutate(agent_gnome,3))
            for i in range(7):
                new_population.append(shuffle_mutate(agent_gnome))
                
        for i in range(len(population))[::-1]:
            del population[i]
        
        population = new_population
            
              
    

In [1]:
population_fitness = []
#population_fitness = genetic_algorithm(5000,1000)